In [1]:
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import cv2
import numpy as np
from loadimages import load_images
import random
from models import FlowNetS
from matplotlib import pyplot as plt

In [2]:
sequences = load_images(1)
random.shuffle(sequences)
sequences = sequences[:500]

In [3]:
print(len(sequences))

500


In [9]:
number_of_epochs = 50
model = FlowNetS()
loss_fn = nn.MSELoss()
# model.load_state_dict(torch.load('./etc.pt))
model.cuda()
learning_rate = 1e-1
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=0.98)

In [ ]:
training_losses = []
validation_losses = []
training_range = int(len(sequences) * 0.8)

print('started training')
for epoch in range(number_of_epochs):
  training_loss = 0.0
  validation_loss = 0.0
  for data in sequences[:training_range]:
    out = model(data[0].unsqueeze(0))
    loss = loss_fn(out.squeeze(0), data[1])
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 6.0)
    training_loss += loss.item()
    optimizer.step()

  for data in sequences[training_range:]:
    out = model(data[0].unsqueeze(0))
    loss = loss_fn(out.squeeze(0), data[1])
    validation_loss += loss.item()

  training_losses.append(training_loss)
  validation_losses.append(validation_loss)
  torch.save(model.state_dict(), './flownets.pt')
  print('Training loss: ' + str(training_loss) + ' ; Validation loss: ' + str(validation_loss) + ' ; Epoch: ' + str(epoch))

started training
Training loss: 2663064.848754883 ; Validation loss: 226446.84274291992 ; Epoch: 0
Training loss: 579313.8387069702 ; Validation loss: 79877.24462890625 ; Epoch: 1
Training loss: 271325.69566345215 ; Validation loss: 52578.01075744629 ; Epoch: 5


In [1]:
plt.plot(training_losses)
plt.plot(validation_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

NameError: name 'plt' is not defined